## 518. Coin Change II
- Description:
  <blockquote>
    You are given an integer array `coins` representing coins of different denominations and an integer `amount` representing a total amount of money.
     
    Return  *the number of combinations that make up that amount* . If that amount of money cannot be made up by any combination of the coins, return `0`.
     
    You may assume that you have an infinite number of each kind of coin.
     
    The answer is **guaranteed** to fit into a signed **32-bit** integer.
     
    **Example 1:**
    **Input:** amount = 5, coins = [1,2,5]
    **Output:** 4
    **Explanation:** there are four ways to make up the amount:
    5=5
    5=2+2+1
    5=2+1+1+1
    5=1+1+1+1+1
     
    **Example 2:**
    **Input:** amount = 3, coins = [2]
    **Output:** 0
    **Explanation:** the amount of 3 cannot be made up just with coins of 2.
     
    **Example 3:**
    **Input:** amount = 10, coins = [10]
    **Output:** 1
     
    **Constraints:**
     
    - `1 <= coins.length <= 300`
    - `1 <= coins[i] <= 5000`
    - All the values of `coins` are **unique**.
    - `0 <= amount <= 5000`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/coin-change-ii/description/)

- Topics: Dynamic Programming

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Top Down DP
For a specific coin, we have two options: either we take this coin and decrease the remaining amount we still need, or we ignore the coin and move to the next one without changing the remaining amount we still need. We add the number of ways to make up the required amount from both choices.

As we may notice, we are breaking down a larger problem into smaller but similar problems, and adding the results of them to get the answer to the original problem. If we choose to take a coin with value, we are now searching for a combination of coins that sum up to amount - value. If we choose to skip the coin, we are still looking for a combination of coins that sum up to amount, but with fewer coins.

As we are breaking down a larger problem into smaller, similar problems, we can use recursion to implement this approach.

Let numberOfWays be the recursive method that we use to solve the problem. It would require two parameters: the index of the current coin under consideration i and the remaining amount needed. It would return the number of ways to make up the amount by only considering the coins beginning from index i.

If amount equals 0, we return 1 as we can always make up the 0 amount by not taking any coins.

If i equals n (where n is the number of coins), we return 0 as we don't have any more coins and hence can't possibly make up the amount. These two form the base cases of our recursive implementation.

We can observe that there are several repeating sub problems in recursion, To avoid this issue, we store the solution of each sub-problem and when we encounter the same subproblem again, we simply refer to the stored result. This is called memoization.


As we know the current state of a sub-problem depends on the index of the current under consideration and the remaining amount, we can use a 2D array memo. memo[i][amount] stores the number of ways to make up amount using the coins beginning from index i.

Here n is the size of coins:
- Time Complexity: O(N * amount)
    - We take O(n⋅amount) to initialize the memo array. There are total of O(n⋅amount) states that are computed. There is no loop inside the recursive method. Each state is computed once and it takes an average of O(1) time to compute the answer of a state using the smaller states. So, it takes O(n⋅amount) time to compute values for all the states.

- Space Complexity: O(N * amount)
    - The memo array takes O(n⋅amount) space.
    - The recursion stack can grow up to a maximum size of O(n+amount) as we are reducing either the number of coins or the required amount while going from one recursive call to another.


In [ ]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        def numberOfWays(i: int, amount: int) -> int:
            if amount == 0:
                return 1
            
            if i == len(coins):
                return 0
            
            if memo[i][amount] != -1:
                return memo[i][amount]

            if coins[i] > amount:
                memo[i][amount] = numberOfWays(i + 1, amount)
            else:
                memo[i][amount] = numberOfWays(i, amount - coins[i]) + numberOfWays(i + 1, amount)
            
            return memo[i][amount]

        # len coins row and len amount+1 columns
        memo = [[-1] * (amount + 1) for _ in range(len(coins))]
        return numberOfWays(0, amount)

### Solution 2, Bottom up DP
We build answers to subproblems iteratively first, then use them to build answers to larger problems.

We create a 2D list dp[n][amount] where dp[i][j] stores the number of ways to make up the j amount using the coins beginning from index i. Note that here, dp[i][j] is equivalent to numberOfWays(i, j) from the previous approach.

We initialize dp[i][0] = 1 for all values of i from 0 to n since we can always make up the amount 0 by not selecting any coins. While moving from bottom to top, this serves as the base case for our solution.

Here n is the size of coins.

    Time complexity: O(n⋅amount).
        We take O(n⋅amount) to initialize the dp array.
        We run two nested to fill the entire dp table. Each state consumes O(1) time to compute. As a result, it takes O(n⋅amount) to fill the dp table using two nested loops.

    Space complexity: O(n⋅amount).
        The dp array takes O(n⋅amount) space.


In [ ]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        n = len(coins)
        dp = [[0] * (amount + 1) for _ in range(n + 1)]
        for i in range(n):
            dp[i][0] = 1

        for i in range(n - 1, -1, -1):
            for j in range(1, amount + 1):
                if coins[i] > j:
                    dp[i][j] = dp[i + 1][j]
                else:
                    dp[i][j] = dp[i + 1][j] + dp[i][j - coins[i]]

        return dp[0][amount]

### Solution 3, Space optimized DP

dp[i][j] = dp[i + 1][j] + dp[i][j - coins[i]]

Looking closely at this transition, we can see that to fill dp[i][j] for a specific i and all values of j we only need the values from dp[i] and dp[i + 1]. Values from older rows like i + 2, i + 3, etc. are no longer relevant.

We can optimize the previous solution by using just one 1D array dp of size amount + 1.

- Time Complexity: O(N)
- Space Complexity: O(N)

In [ ]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        n = len(coins)
        dp = [0] * (amount + 1)
        dp[0] = 1

        for i in range(n - 1, -1, -1):
            for j in range(coins[i], amount + 1):
                dp[j] += dp[j - coins[i]]

        return dp[amount]

Great! Let's walk through how the DP table fills step by step for the example:

    amount = 5
    coins = [1, 2, 5]

We initialize:

dp = [1, 0, 0, 0, 0, 0]
# dp[0] = 1 (one way to make amount 0: choose nothing)


🔁 Step 1: Process coin = 1

We update dp[i] += dp[i - 1] for i from 1 to 5:

    i=1: dp[1] += dp[0] → 0 + 1 = 1
    i=2: dp[2] += dp[1] → 0 + 1 = 1
    i=3: dp[3] += dp[2] → 0 + 1 = 1
    i=4: dp[4] += dp[3] → 0 + 1 = 1
    i=5: dp[5] += dp[4] → 0 + 1 = 1

Now:

dp = [1, 1, 1, 1, 1, 1]

Apply Code

→ This means: using only coin 1, there's exactly 1 way to make every amount (all 1s).

🔁 Step 2: Process coin = 2

Update dp[i] += dp[i - 2] for i from 2 to 5:

    i=2: dp[2] += dp[0] → 1 + 1 = 2
    i=3: dp[3] += dp[1] → 1 + 1 = 2
    i=4: dp[4] += dp[2] → 1 + 2 = 3
    i=5: dp[5] += dp[3] → 1 + 2 = 3

Now:

dp = [1, 1, 2, 2, 3, 3]

Apply Code

Interpretation:

    Amount 2: {1+1, 2} → 2 ways ✅
    Amount 3: {1+1+1, 1+2} → 2 ways ✅
    Amount 4: {1×4, 1×2+2, 2+2} → 3 ways ✅
    Amount 5: {1×5, 1×3+2, 1+2×2} → 3 ways so far (we’ll add more with coin 5)

🔁 Step 3: Process coin = 5

Update dp[i] += dp[i - 5] for i from 5 to 5:

    i=5: dp[5] += dp[0] → 3 + 1 = 4

Final:

dp = [1, 1, 2, 2, 3, 4]

Apply Code

So, dp[5] = 4 — the answer!

The new combination added is just {5}.

In [ ]:
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        dp = [0] * (amount + 1)
        dp[0] = 1  # One way to make 0: use no coins

        for coin in coins:
            for i in range(coin, amount + 1):
                dp[i] += dp[i - coin]

        return dp[amount]